In [ ]:
from openai import OpenAI
from PIL import Image
import os
import pandas as pd
import base64
import requests
import cv2 as cv
import numpy as np
from collections import Counter
from io import BytesIO

In [ ]:
# library for google sheet access
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()


In [ ]:
# mount my google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Fill in your OpenAI API key

In [ ]:
openai_api_key=""

System Prompts

In [ ]:
import pandas as pd
import json
import requests
import base64
from io import BytesIO
from PIL import Image
import time

def display_image_from_url(image_url):
    try:
        response = requests.get(image_url)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))
        display(image)
    except Exception as e:
        print(f"Failed to display image: {e}")



def build_full_system_prompt():

    return """### Role
You are a high-precision Data Extraction Specialist for the vape/cannabis industry. Your task is to extract flavor names from product images with 100% literal accuracy.

### Critical Rules (Expert Constraints)
1. **Literal Extraction Only**: Extract flavor descriptors EXACTLY as they appear on the package.
   - NO spell correction (e.g., keep "Strazzberry", do not change to "Strawberry").
   - NO background inference (e.g., ignore fruit/cereal images if text isn't present).
   - Strict Verbatim: Do not translate, do not "fix", and do not assume. If the text says "STAR BUZZ", you MUST output "STAR BUZZ". Do not output "Strawberry" or "Starfruit" based on your guess.
2. **Clean Output**: Each flavor string must be "naked".
   - NO bullet points (-), NO stars (*), NO whitespace padding.
   - NO parentheses or metadata (e.g., do not include "(blue device)").
3. **Exclude Nicotine**: If nicotine content (e.g., "5%", "50mg") appears next to the flavor, DO NOT include it in the flavor field.
4. **Handling Absence**: If no flavor text is visible, the list must be ["missing"].
5. **Brand Identification**: Identify the brand associated with each flavor if multiple brands exist.
6. **Anti-Hallucination Rule**: If you see a word you don't recognize as a "standard" flavor (e.g., "Star Buzz"), you MUST extract it exactly as written. NEVER replace a literal word with a "common" flavor name.

### Output Schema (JSON)
Return ONLY a JSON object:
- "flavors_list": Array of strings. Pure flavor text only. For example, ["Flavor1", "Flavor2"]
- "multiple_descriptors": "1" (if >1), "0" (if 1), "n/a" (if none flavor descriptor extracted due to poor image quality or no flavor shown on the image).
- "brand_name": Brand name found.
- "extraction_evidence": Internal note on text location and ignored art.
"""

def analyze_and_explode(image_url,  openai_api_key,max_retries=5):

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai_api_key}"
    }

    payload = {
        "model": "gpt-4.1",
        "messages": [
            {"role": "system", "content": build_full_system_prompt()},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": f"Analyze this product and return a json object."},
                    {"type": "image_url", "image_url": {"url": image_url}}
                ]
            }
        ],
        "response_format": { "type": "json_object" },
        "max_tokens": 1000
    }


    for attempt in range(max_retries):
        try:
            response = requests.post("https://api.openai.com/v1/chat/completions",
                                     headers=headers, json=payload)
            response.raise_for_status()
            response_json = response.json()



            if 'choices' in response_json:
                return response_json['choices'][0]['message']['content']
            else:
                return "No information"

        except Exception:
            time.sleep(2)
            continue

    return "No information"




Batch Processing:
1. Modify sheet_url to your own CSV file
2. Modigy worksheet to your sheet name
3. df_test['livelink'] is the column save all Image URLs, change it to your column saving URLs

In [ ]:
# Load CSV file
gc = gspread.authorize(creds)
sheet_url = "https://docs.google.com/spreadsheets/d/1dH7VKiGKhdP7lxA5R2fPjkh6f1LawwgCjr-nuvn918s/edit?usp=sharing"
sh = gc.open_by_url(sheet_url)
worksheet = sh.worksheet('dash space and star')
df_test = pd.DataFrame(worksheet.get_all_records())


all_rows = []

for i in range(len(df_test)):

    image_url = df_test['livelink'].to_list()[i]

    print(f"--- Processing {i}th Image ---")
    display_image_from_url(image_url)

    llm_result = analyze_and_explode(image_url,openai_api_key)
    print(llm_result)


    if llm_result != "No information":
        try:

            result_dict = json.loads(llm_result)


            if isinstance(result_dict.get('flavors_list'), list):
                result_dict['flavor_descriptor'] = ", ".join(result_dict['flavors_list'])
            else:
                result_dict['flavor_descriptor'] = result_dict.get('flavors_list', "missing")




            all_rows.append(result_dict)


        except Exception as e:
            print(f"Error parsing JSON at index {i}: {e}")
            print(f"Raw LLM output: {llm_result}")
    else:
        print(f"Skipping {i}th image due to 'No information'.")


df_final_output = pd.DataFrame(all_rows)


print("\n--- Final Results (Single Row per Image) ---")
df_final_output.head()